In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Set up enviroment
from google.cloud import bigquery

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("broadstreet_adi", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
print('bigquery client setup complete')

Using Kaggle's public dataset BigQuery integration.
bigquery client setup complete


In [3]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

area_deprivation_index_by_census_block_group
area_deprivation_index_by_county
area_deprivation_index_by_zipcode


In [4]:
ADI_country_table = dataset_ref.table("area_deprivation_index_by_census_block_group")
#API request
Table = client.get_table(ADI_country_table)

In [5]:
Table.schema

[SchemaField('geo_id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('state_fips_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('county_fips_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('block_group_fips_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('description', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('county_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('state_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('state', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('year', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('area_deprivation_index_percent', 'FLOAT', 'NULLABLE', None, (), None)]

In [6]:
client.list_rows(Table,max_results =5 ).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,geo_id,state_fips_code,county_fips_code,block_group_fips_code,description,county_name,state_name,state,year,area_deprivation_index_percent
0,15000US010010201001,01,01001,010010201001,"Block Group 1, Census Tract 201",Autauga County,Alabama,AL,2018,79.0
1,15000US010010201002,01,01001,010010201002,"Block Group 2, Census Tract 201",Autauga County,Alabama,AL,2018,30.0
2,15000US010010202001,01,01001,010010202001,"Block Group 1, Census Tract 202",Autauga County,Alabama,AL,2018,76.0
3,15000US010010202002,01,01001,010010202002,"Block Group 2, Census Tract 202",Autauga County,Alabama,AL,2018,66.0
4,15000US010010203001,01,01001,010010203001,"Block Group 1, Census Tract 203",Autauga County,Alabama,AL,2018,42.0


In [7]:
query = """
        SELECT geo_id,state_fips_code, county_fips_code,county_name, state_name, state, year, area_deprivation_index_percent
        FROM `bigquery-public-data.broadstreet_adi.area_deprivation_index_by_census_block_group`
        WHERE area_deprivation_index_percent BETWEEN 10.0 AND 90.0
        """
query_job = client.query(query, job_config=safe_config)
# API request - run the query, and return a pandas DataFrame
API = query_job.to_dataframe()

# View top few rows of results
print(API.head())

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


                geo_id state_fips_code county_fips_code     county_name  \
0  15000US010010201001              01            01001  Autauga County   
1  15000US010010201002              01            01001  Autauga County   
2  15000US010010202001              01            01001  Autauga County   
3  15000US010010202002              01            01001  Autauga County   
4  15000US010010203001              01            01001  Autauga County   

  state_name state  year  area_deprivation_index_percent  
0    Alabama    AL  2018                            79.0  
1    Alabama    AL  2018                            30.0  
2    Alabama    AL  2018                            76.0  
3    Alabama    AL  2018                            66.0  
4    Alabama    AL  2018                            42.0  


In [11]:
#API.to_csv('Area_deprivation_scoreNew.csv', index=False)